In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn. metrics import accuracy_score
from sklearn import metrics
import matplotlib.pyplot as plt
import joblib
from pathlib import Path
import pickle
from datetime import date
import datetime as dt
from patsy import dmatrices
import statsmodels.api as sm
from collections import defaultdict
import copy
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:

TRMMPath = Path('InputData/TRMM&GLDASdata/TRMM')
GLDASPath = Path('InputData/TRMM&GLDASdata/GLDAS')

#load historical streamflows from BCC, LCC, CC, PC
Wheat = pd.read_excel('InputData/District_Wheat_1982_2017.xlsx', sheet_name = 'Ton_Hec')
TRMM = {i.stem[0:30] : pd.read_csv(i) for i in TRMMPath.glob('**/*.csv')}
GLDAS = {i.stem[0:30] : pd.read_csv(i) for i in GLDASPath.glob('**/*.csv')}

#set Wheat data
Wheat=Wheat.set_index('Year')
Wheat=Wheat.loc[2001:]


#select and rename columns
GLDAS_col = ['Tair_f_inst','SoilMoi0_10cm_inst','SoilMoi10_40cm_inst',
                          'SoilMoi40_100cm_inst','SoilTMP0_10cm_inst','SoilTMP10_40cm_inst']

GLDAS_renamecol =['Air_temp_C', 'SM_0_10cm','SM_10_40cm','SM_40_100cm','STemp_0_10cm','STemp_10_40cm']

TRMM_col = ['precipitation']
TRMM_renamecol = ['Precip']

#Put Everyting in the correct date format
for i in GLDAS:
    
    GLDAS[i] = GLDAS[i].rename(columns = {'system:time_start':'Date'})
    GLDAS[i].index = pd.to_datetime(GLDAS[i]['Date'])
    GLDAS[i] = GLDAS[i][GLDAS_col]
    GLDAS[i].columns=GLDAS_renamecol
    
    #create date range to remove
    yr = i[-4:]
    start_date = str(yr)+'-4-21'
    end_date = str(yr)+'-10-14'
    Dt_range = pd.date_range(start = start_date, end = end_date)
    GLDAS[i] = GLDAS[i].loc[~GLDAS[i].index.isin(Dt_range)]
    
    #need to convert air temp from k to C
    GLDAS[i]['Air_temp_C'] = GLDAS[i]['Air_temp_C']-273.15 
    GLDAS[i]['STemp_0_10cm'] = GLDAS[i]['STemp_0_10cm']-273.15 
    GLDAS[i]['STemp_10_40cm'] = GLDAS[i]['STemp_10_40cm']-273.15 
    
#make GLDAS keys the same as TRMM
GLDASkeys = list(GLDAS.keys())
for i in GLDASkeys:
    #make new key 1 year later
    district = i[:3]
    year = i[-5:]
    new_key = district+year
    GLDAS[new_key] = GLDAS.pop(i)
    
for i in TRMM:
    TRMM[i] = TRMM[i].rename(columns = {'system:time_start':'Date'})
    TRMM[i].index = pd.to_datetime(TRMM[i]['Date'])
    TRMM[i] = TRMM[i][TRMM_col]
    TRMM[i].columns=TRMM_renamecol
    
     #create date range to remove
    yr = i[-4:]
    start_date = str(yr)+'-4-21'
    end_date = str(yr)+'-10-14'
    Dt_range = pd.date_range(start = start_date, end = end_date)
    TRMM[i] = TRMM[i].loc[~TRMM[i].index.isin(Dt_range)]


In [3]:
#create rabi season datasets
GLDAS_Fall = copy.deepcopy(GLDAS)
GLDAS_Spring = copy.deepcopy(GLDAS)

for i in GLDAS:
    #create date range to remove
    yr = i[-4:]
    end_date2 = str(yr)+'-4-21'
    start_date2 = str(yr)+'-10-14'
    
    GLDAS_Fall[i]=GLDAS_Fall[i].loc[start_date2:]
    GLDAS_Spring[i]=GLDAS_Spring[i].loc[:end_date2]
    
#need to run in reverse to not overlap years
for i in reversed(GLDAS_Fall):
    #make new key 1 year later
    yrmod = str(int(i[-4:])+1)
    keyinfo = i[:-4]
    new_key = keyinfo+yrmod
    GLDAS_Fall[new_key] = GLDAS_Fall.pop(i)
    
GLDAS_Rabi = copy.deepcopy(GLDAS_Fall)
#need to drop 2018 Rabi values and 2000 Spring values as they are not complete years
dropRabi = ['Raw_2018' , 'Jhe_2018', 'Cha_2018', 'Att_2018']
dropSpring = ['Raw_2000' , 'Jhe_2000', 'Cha_2000', 'Att_2000']

[GLDAS_Rabi.pop(key) for key in dropRabi]
[GLDAS_Spring.pop(key) for key in dropSpring]

for i in GLDAS_Rabi:
    GLDAS_Rabi[i] = GLDAS_Rabi[i].append(GLDAS_Spring[i])

In [4]:
#create rabi season datasets
TRMM_Fall = copy.deepcopy(TRMM)
TRMM_Spring = copy.deepcopy(TRMM)

for i in TRMM:
    #create date range to remove
    yr = i[-4:]
    end_date2 = str(yr)+'-4-21'
    start_date2 = str(yr)+'-10-14'
    
    TRMM_Fall[i]=TRMM_Fall[i].loc[start_date2:]
    TRMM_Spring[i]=TRMM_Spring[i].loc[:end_date2]
    
#need to run in reverse to not overlap years
for i in reversed(TRMM_Fall):
    #make new key 1 year later
    yrmod = str(int(i[-4:])+1)
    keyinfo = i[:-4]
    new_key = keyinfo+yrmod
    TRMM_Fall[new_key] = TRMM_Fall.pop(i)
    
TRMM_Rabi = copy.deepcopy(TRMM_Fall)
#need to drop 2018 Rabi values and 2000 Spring values as they are not complete years
dropRabi = ['Raw_2018' , 'Jhe_2018', 'Cha_2018', 'Att_2018', 'Raw_2000' , 'Jhe_2000', 'Cha_2000', 'Att_2000']
dropSpring = ['Raw_1999' , 'Jhe_1999', 'Cha_1999', 'Att_1999','Raw_2000' , 'Jhe_2000', 'Cha_2000', 'Att_2000']

[TRMM_Rabi.pop(key) for key in dropRabi]
[TRMM_Spring.pop(key) for key in dropSpring]

for i in TRMM_Rabi:
    TRMM_Rabi[i] = TRMM_Rabi[i].append(TRMM_Spring[i])

In [5]:
#create all of the GLDAS_Rabi_Sums temperature features
GLDAS_Rabi_Sums=copy.deepcopy(GLDAS_Rabi)

for i in GLDAS_Rabi_Sums:
    #Air temperature
    
    #Find all hours above 5C, and multiply by temporal resolution (3hrs)
    GLDAS_Rabi_Sums[i]['Hrsabove_5C'] = 0
    GLDAS_Rabi_Sums[i].loc[GLDAS_Rabi_Sums[i]['Air_temp_C'] > 5, 'Hrsabove_5C'] = 1
    GLDAS_Rabi_Sums[i]['Hrsabove_5C']=GLDAS_Rabi_Sums[i]['Hrsabove_5C']*3
    
    #Find all hours above 10C, and multiply by temporal resolution (3hrs)
    GLDAS_Rabi_Sums[i]['Hrsabove_10C'] = 0
    GLDAS_Rabi_Sums[i].loc[GLDAS_Rabi_Sums[i]['Air_temp_C'] > 10, 'Hrsabove_10C'] = 1
    GLDAS_Rabi_Sums[i]['Hrsabove_10C']=GLDAS_Rabi_Sums[i]['Hrsabove_10C']*3
    
    #Find all hours above 15C, and multiply by temporal resolution (3hrs)
    GLDAS_Rabi_Sums[i]['Hrsabove_15C'] = 0
    GLDAS_Rabi_Sums[i].loc[GLDAS_Rabi_Sums[i]['Air_temp_C'] > 15, 'Hrsabove_15C'] = 1
    GLDAS_Rabi_Sums[i]['Hrsabove_15C']=GLDAS_Rabi_Sums[i]['Hrsabove_15C']*3
    
    #Find all hours above 20C, and multiply by temporal resolution (3hrs)
    GLDAS_Rabi_Sums[i]['Hrsabove_20C'] = 0
    GLDAS_Rabi_Sums[i].loc[GLDAS_Rabi_Sums[i]['Air_temp_C'] > 20, 'Hrsabove_20C'] = 1
    GLDAS_Rabi_Sums[i]['Hrsabove_20C']=GLDAS_Rabi_Sums[i]['Hrsabove_20C']*3
    
    #Find all hours above 20C, and multiply by temporal resolution (3hrs)
    GLDAS_Rabi_Sums[i]['Hrsabove_25C'] = 0
    GLDAS_Rabi_Sums[i].loc[GLDAS_Rabi_Sums[i]['Air_temp_C'] > 25, 'Hrsabove_25C'] = 1
    GLDAS_Rabi_Sums[i]['Hrsabove_25C']=GLDAS_Rabi_Sums[i]['Hrsabove_25C']*3
    
    
    #Find all hours below 0C, and multiply by temporal resolution (3hrs)
    GLDAS_Rabi_Sums[i]['Hrsbelow0C'] = 0
    GLDAS_Rabi_Sums[i].loc[GLDAS_Rabi_Sums[i]['Air_temp_C'] <=0, 'Hrsbelow0C'] = 1
    GLDAS_Rabi_Sums[i]['Hrsbelow0C']=GLDAS_Rabi_Sums[i]['Hrsbelow0C']*3
    
    #Find hours below 5c
    GLDAS_Rabi_Sums[i]['Hrsbelow5C'] = 0
    GLDAS_Rabi_Sums[i].loc[GLDAS_Rabi_Sums[i]['Air_temp_C'] <5, 'Hrsbelow5C'] = 1
    GLDAS_Rabi_Sums[i]['Hrsbelow5C']=GLDAS_Rabi_Sums[i]['Hrsbelow5C']*3
    
    #Find hours below 10c
    GLDAS_Rabi_Sums[i]['Hrsbelow10C'] = 0
    GLDAS_Rabi_Sums[i].loc[GLDAS_Rabi_Sums[i]['Air_temp_C'] <10, 'Hrsbelow10C'] = 1
    GLDAS_Rabi_Sums[i]['Hrsbelow10C']=GLDAS_Rabi_Sums[i]['Hrsbelow10C']*3
    
    #Find hours below 15c
    GLDAS_Rabi_Sums[i]['Hrsbelow15C'] = 0
    GLDAS_Rabi_Sums[i].loc[GLDAS_Rabi_Sums[i]['Air_temp_C'] <15, 'Hrsbelow15C'] = 1
    GLDAS_Rabi_Sums[i]['Hrsbelow15C']=GLDAS_Rabi_Sums[i]['Hrsbelow15C']*3
    
    #Find hours between 0-5C
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs0_5C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['Air_temp_C'] >0) & (GLDAS_Rabi_Sums[i]['Air_temp_C'] <5), 'Air_temp_Hrs0_5C'] = 1
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs0_5C']=GLDAS_Rabi_Sums[i]['Air_temp_Hrs0_5C']*3
    
    #Find hours between 5-10C
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs5_10C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['Air_temp_C'] >5) & (GLDAS_Rabi_Sums[i]['Air_temp_C'] <10), 'Air_temp_Hrs5_10C'] = 1
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs5_10C']=GLDAS_Rabi_Sums[i]['Air_temp_Hrs5_10C']*3
    
    #Find hours between 10-15C
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs10_15C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['Air_temp_C'] >10) & (GLDAS_Rabi_Sums[i]['Air_temp_C'] <15), 'Air_temp_Hrs10_15C'] = 1
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs10_15C']=GLDAS_Rabi_Sums[i]['Air_temp_Hrs10_15C']*3
    
    #Find hours between 15-20C
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs15_20C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['Air_temp_C'] >15) & (GLDAS_Rabi_Sums[i]['Air_temp_C'] <20), 'Air_temp_Hrs15_20C'] = 1
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs15_20C']=GLDAS_Rabi_Sums[i]['Air_temp_Hrs15_20C']*3
    
    #Find hours between 20-25C
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs20_25C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['Air_temp_C'] >20) & (GLDAS_Rabi_Sums[i]['Air_temp_C'] <25), 'Air_temp_Hrs20_25C'] = 1
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs20_25C']=GLDAS_Rabi_Sums[i]['Air_temp_Hrs20_25C']*3
    
    #Find hours between 0-10C
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs0_10C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['Air_temp_C'] >0) & (GLDAS_Rabi_Sums[i]['Air_temp_C'] <10), 'Air_temp_Hrs0_10C'] = 1
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs0_10C']=GLDAS_Rabi_Sums[i]['Air_temp_Hrs0_10C']*3
    
    #Find hours between 5-15C
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs5_15C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['Air_temp_C'] >5) & (GLDAS_Rabi_Sums[i]['Air_temp_C'] <15), 'Air_temp_Hrs5_15C'] = 1
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs5_15C']=GLDAS_Rabi_Sums[i]['Air_temp_Hrs5_15C']*3
    
    #Find hours between 10-20C
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs10_20C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['Air_temp_C'] >10) & (GLDAS_Rabi_Sums[i]['Air_temp_C'] <20), 'Air_temp_Hrs10_20C'] = 1
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs10_20C']=GLDAS_Rabi_Sums[i]['Air_temp_Hrs10_20C']*3
    
    #Find hours between 15-25C
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs15_25C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['Air_temp_C'] >15) & (GLDAS_Rabi_Sums[i]['Air_temp_C'] <25), 'Air_temp_Hrs15_25C'] = 1
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs15_25C']=GLDAS_Rabi_Sums[i]['Air_temp_Hrs15_25C']*3
    
    #Find hours between 5-20C
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs5_20C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['Air_temp_C'] >5) & (GLDAS_Rabi_Sums[i]['Air_temp_C'] <20), 'Air_temp_Hrs15_25C'] = 1
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs5_20C']=GLDAS_Rabi_Sums[i]['Air_temp_Hrs5_20C']*3
    
    #Find hours between 10-25C
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs10_25C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['Air_temp_C'] >10) & (GLDAS_Rabi_Sums[i]['Air_temp_C'] <25), 'Air_temp_Hrs10_25C'] = 1
    GLDAS_Rabi_Sums[i]['Air_temp_Hrs10_25C']=GLDAS_Rabi_Sums[i]['Air_temp_Hrs10_25C']*3
    
    #Soil Temperature
    #Find all hours above 5C, and multiply by temporal resolution (3hrs)
    GLDAS_Rabi_Sums[i]['Hrsabove_5C'] = 0
    GLDAS_Rabi_Sums[i].loc[GLDAS_Rabi_Sums[i]['STemp_0_10cm'] > 5, 'Hrsabove_5C'] = 1
    GLDAS_Rabi_Sums[i]['Hrsabove_5C']=GLDAS_Rabi_Sums[i]['Hrsabove_5C']*3
    
    #Find all hours above 10C, and multiply by temporal resolution (3hrs)
    GLDAS_Rabi_Sums[i]['Hrsabove_10C'] = 0
    GLDAS_Rabi_Sums[i].loc[GLDAS_Rabi_Sums[i]['STemp_0_10cm'] > 10, 'Hrsabove_10C'] = 1
    GLDAS_Rabi_Sums[i]['Hrsabove_10C']=GLDAS_Rabi_Sums[i]['Hrsabove_10C']*3
    
    #Find all hours above 15C, and multiply by temporal resolution (3hrs)
    GLDAS_Rabi_Sums[i]['Hrsabove_15C'] = 0
    GLDAS_Rabi_Sums[i].loc[GLDAS_Rabi_Sums[i]['STemp_0_10cm'] > 15, 'Hrsabove_15C'] = 1
    GLDAS_Rabi_Sums[i]['Hrsabove_15C']=GLDAS_Rabi_Sums[i]['Hrsabove_15C']*3
    
    #Find all hours above 20C, and multiply by temporal resolution (3hrs)
    GLDAS_Rabi_Sums[i]['Hrsabove_20C'] = 0
    GLDAS_Rabi_Sums[i].loc[GLDAS_Rabi_Sums[i]['STemp_0_10cm'] > 20, 'Hrsabove_20C'] = 1
    GLDAS_Rabi_Sums[i]['Hrsabove_20C']=GLDAS_Rabi_Sums[i]['Hrsabove_20C']*3
    
    #Find all hours above 20C, and multiply by temporal resolution (3hrs)
    GLDAS_Rabi_Sums[i]['Hrsabove_25C'] = 0
    GLDAS_Rabi_Sums[i].loc[GLDAS_Rabi_Sums[i]['STemp_0_10cm'] > 25, 'Hrsabove_25C'] = 1
    GLDAS_Rabi_Sums[i]['Hrsabove_25C']=GLDAS_Rabi_Sums[i]['Hrsabove_25C']*3
    
    
    #Find all hours below 0C, and multiply by temporal resolution (3hrs)
    GLDAS_Rabi_Sums[i]['Hrsbelow0C'] = 0
    GLDAS_Rabi_Sums[i].loc[GLDAS_Rabi_Sums[i]['STemp_0_10cm'] <=0, 'Hrsbelow0C'] = 1
    GLDAS_Rabi_Sums[i]['Hrsbelow0C']=GLDAS_Rabi_Sums[i]['Hrsbelow0C']*3
    
    #Find hours below 5c
    GLDAS_Rabi_Sums[i]['Hrsbelow5C'] = 0
    GLDAS_Rabi_Sums[i].loc[GLDAS_Rabi_Sums[i]['STemp_0_10cm'] <5, 'Hrsbelow5C'] = 1
    GLDAS_Rabi_Sums[i]['Hrsbelow5C']=GLDAS_Rabi_Sums[i]['Hrsbelow5C']*3
    
    #Find hours below 10c
    GLDAS_Rabi_Sums[i]['Hrsbelow10C'] = 0
    GLDAS_Rabi_Sums[i].loc[GLDAS_Rabi_Sums[i]['STemp_0_10cm'] <10, 'Hrsbelow10C'] = 1
    GLDAS_Rabi_Sums[i]['Hrsbelow10C']=GLDAS_Rabi_Sums[i]['Hrsbelow10C']*3
    
    #Find hours below 15c
    GLDAS_Rabi_Sums[i]['Hrsbelow15C'] = 0
    GLDAS_Rabi_Sums[i].loc[GLDAS_Rabi_Sums[i]['STemp_0_10cm'] <15, 'Hrsbelow15C'] = 1
    GLDAS_Rabi_Sums[i]['Hrsbelow15C']=GLDAS_Rabi_Sums[i]['Hrsbelow15C']*3
    
    #Find hours between 0-5C
    GLDAS_Rabi_Sums[i]['STemp_Hrs0_5C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['STemp_0_10cm'] >0) & (GLDAS_Rabi_Sums[i]['STemp_0_10cm'] <5), 'STemp_Hrs0_5C'] = 1
    GLDAS_Rabi_Sums[i]['STemp_Hrs0_5C']=GLDAS_Rabi_Sums[i]['STemp_Hrs0_5C']*3
    
    #Find hours between 5-10C
    GLDAS_Rabi_Sums[i]['STemp_Hrs5_10C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['STemp_0_10cm'] >5) & (GLDAS_Rabi_Sums[i]['STemp_0_10cm'] <10), 'STemp_Hrs5_10C'] = 1
    GLDAS_Rabi_Sums[i]['STemp_Hrs5_10C']=GLDAS_Rabi_Sums[i]['STemp_Hrs5_10C']*3
    
    #Find hours between 10-15C
    GLDAS_Rabi_Sums[i]['STemp_Hrs10_15C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['STemp_0_10cm'] >10) & (GLDAS_Rabi_Sums[i]['STemp_0_10cm'] <15), 'STemp_Hrs10_15C'] = 1
    GLDAS_Rabi_Sums[i]['STemp_Hrs10_15C']=GLDAS_Rabi_Sums[i]['STemp_Hrs10_15C']*3
    
    #Find hours between 15-20C
    GLDAS_Rabi_Sums[i]['STemp_Hrs15_20C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['STemp_0_10cm'] >15) & (GLDAS_Rabi_Sums[i]['STemp_0_10cm'] <20), 'STemp_Hrs15_20C'] = 1
    GLDAS_Rabi_Sums[i]['STemp_Hrs15_20C']=GLDAS_Rabi_Sums[i]['STemp_Hrs15_20C']*3
    
    #Find hours between 20-25C
    GLDAS_Rabi_Sums[i]['STemp_Hrs20_25C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['STemp_0_10cm'] >20) & (GLDAS_Rabi_Sums[i]['STemp_0_10cm'] <25), 'STemp_Hrs20_25C'] = 1
    GLDAS_Rabi_Sums[i]['STemp_Hrs20_25C']=GLDAS_Rabi_Sums[i]['STemp_Hrs20_25C']*3
    
    #Find hours between 0-10C
    GLDAS_Rabi_Sums[i]['STemp_Hrs0_10C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['STemp_0_10cm'] >0) & (GLDAS_Rabi_Sums[i]['STemp_0_10cm'] <10), 'STemp_Hrs0_10C'] = 1
    GLDAS_Rabi_Sums[i]['STemp_Hrs0_10C']=GLDAS_Rabi_Sums[i]['STemp_Hrs0_10C']*3
    
    #Find hours between 5-15C
    GLDAS_Rabi_Sums[i]['STemp_Hrs5_15C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['STemp_0_10cm'] >5) & (GLDAS_Rabi_Sums[i]['STemp_0_10cm'] <15), 'STemp_Hrs5_15C'] = 1
    GLDAS_Rabi_Sums[i]['STemp_Hrs5_15C']=GLDAS_Rabi_Sums[i]['STemp_Hrs5_15C']*3
    
    #Find hours between 10-20C
    GLDAS_Rabi_Sums[i]['STemp_Hrs10_20C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['STemp_0_10cm'] >10) & (GLDAS_Rabi_Sums[i]['STemp_0_10cm'] <20), 'STemp_Hrs10_20C'] = 1
    GLDAS_Rabi_Sums[i]['STemp_Hrs10_20C']=GLDAS_Rabi_Sums[i]['STemp_Hrs10_20C']*3
    
    #Find hours between 15-25C
    GLDAS_Rabi_Sums[i]['STemp_Hrs15_25C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['STemp_0_10cm'] >15) & (GLDAS_Rabi_Sums[i]['STemp_0_10cm'] <25), 'STemp_Hrs15_25C'] = 1
    GLDAS_Rabi_Sums[i]['STemp_Hrs15_25C']=GLDAS_Rabi_Sums[i]['STemp_Hrs15_25C']*3
    
    #Find hours between 5-20C
    GLDAS_Rabi_Sums[i]['STemp_Hrs5_20C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['STemp_0_10cm'] >5) & (GLDAS_Rabi_Sums[i]['STemp_0_10cm'] <20), 'STemp_Hrs15_25C'] = 1
    GLDAS_Rabi_Sums[i]['STemp_Hrs5_20C']=GLDAS_Rabi_Sums[i]['STemp_Hrs5_20C']*3
    
    #Find hours between 10-25C
    GLDAS_Rabi_Sums[i]['STemp_Hrs10_25C'] = 0
    GLDAS_Rabi_Sums[i].loc[(GLDAS_Rabi_Sums[i]['STemp_0_10cm'] >10) & (GLDAS_Rabi_Sums[i]['STemp_0_10cm'] <25), 'STemp_Hrs10_25C'] = 1
    GLDAS_Rabi_Sums[i]['STemp_Hrs10_25C']=GLDAS_Rabi_Sums[i]['STemp_Hrs10_25C']*3
    

In [6]:
#create precipitation metrics
precipday = 0.1

for i in TRMM_Rabi:
    #hrs of measurable precipitation
    TRMM_Rabi[i]['PrecipHrs'] = 0
    TRMM_Rabi[i].loc[TRMM_Rabi[i]['Precip'] >precipday, 'PrecipHrs'] = 1
    TRMM_Rabi[i]['PrecipHrs']=TRMM_Rabi[i]['PrecipHrs']*3
    
    #Precipitation > 2mm/hr
    TRMM_Rabi[i]['Precip_above2mmhr'] = 0
    TRMM_Rabi[i].loc[TRMM_Rabi[i]['Precip'] >2, 'Precip_above2mmhr'] = 1
    TRMM_Rabi[i]['Precip_above2mmhr']=TRMM_Rabi[i]['Precip_above2mmhr']*3
    
    #Precipitation > 5mm/hr
    TRMM_Rabi[i]['Precip_above5mmhr'] = 0
    TRMM_Rabi[i].loc[TRMM_Rabi[i]['Precip'] >5, 'Precip_above5mmhr'] = 1
    TRMM_Rabi[i]['Precip_above5mmhr']=TRMM_Rabi[i]['Precip_above5mmhr']*3
    
    #Precipitation > 8mm/hr
    TRMM_Rabi[i]['Precip_above8mmhr'] = 0
    TRMM_Rabi[i].loc[TRMM_Rabi[i]['Precip'] >8, 'Precip_above8mmhr'] = 1
    TRMM_Rabi[i]['Precip_above8mmhr']=TRMM_Rabi[i]['Precip_above8mmhr']*3
    
    #Precipitation > 12mm/hr
    TRMM_Rabi[i]['Precip_above12mmhr'] = 0
    TRMM_Rabi[i].loc[TRMM_Rabi[i]['Precip'] >12, 'Precip_above12mmhr'] = 1
    TRMM_Rabi[i]['Precip_above12mmhr']=TRMM_Rabi[i]['Precip_above12mmhr']*3
    
    #Precipitation > 15mm/hr
    TRMM_Rabi[i]['Precip_above15mmhr'] = 0
    TRMM_Rabi[i].loc[TRMM_Rabi[i]['Precip'] >15, 'Precip_above15mmhr'] = 1
    TRMM_Rabi[i]['Precip_above15mmhr']=TRMM_Rabi[i]['Precip_above15mmhr']*3
    
    #Find hoursless than 8mmhr
    TRMM_Rabi[i]['Precip_below8mmhr'] = 0
    TRMM_Rabi[i].loc[(TRMM_Rabi[i]['Precip'] > precipday) & (TRMM_Rabi[i]['Precip'] <8), 'Precip_below8mmhr'] = 1
    TRMM_Rabi[i]['Precip_below8mmhr']=TRMM_Rabi[i]['Precip_below8mmhr']*3
    
    #Find hoursless than 12mmhr
    TRMM_Rabi[i]['Precip_below12mmhr'] = 0
    TRMM_Rabi[i].loc[(TRMM_Rabi[i]['Precip'] > precipday) & (TRMM_Rabi[i]['Precip'] <12), 'Precip_below12mmhr'] = 1
    TRMM_Rabi[i]['Precip_below12mmhr']=TRMM_Rabi[i]['Precip_below12mmhr']*3
    
    #Find hoursless than 15mmhr
    TRMM_Rabi[i]['Precip_below15mmhr'] = 0
    TRMM_Rabi[i].loc[(TRMM_Rabi[i]['Precip'] > precipday) & (TRMM_Rabi[i]['Precip'] <15), 'Precip_below15mmhr'] = 1
    TRMM_Rabi[i]['Precip_below15mmhr']=TRMM_Rabi[i]['Precip_below15mmhr']*3
    
     #Find hoursless between 1-5mmhr
    TRMM_Rabi[i]['Precip_1_5mmhr'] = 0
    TRMM_Rabi[i].loc[(TRMM_Rabi[i]['Precip'] > 1) & (TRMM_Rabi[i]['Precip'] <5), 'Precip_1_5mmhr'] = 1
    TRMM_Rabi[i]['Precip_1_5mmhr']=TRMM_Rabi[i]['Precip_1_5mmhr']*3
    
     #Find hoursless between 5-8mmhr
    TRMM_Rabi[i]['Precip_5_8mmhr'] = 0
    TRMM_Rabi[i].loc[(TRMM_Rabi[i]['Precip'] > 5) & (TRMM_Rabi[i]['Precip'] <8), 'Precip_5_8mmhr'] = 1
    TRMM_Rabi[i]['Precip_5_8mmhr']=TRMM_Rabi[i]['Precip_5_8mmhr']*3
    
    #Find hoursless between 8-12mmhr
    TRMM_Rabi[i]['Precip_8_12mmhr'] = 0
    TRMM_Rabi[i].loc[(TRMM_Rabi[i]['Precip'] > 8) & (TRMM_Rabi[i]['Precip'] <12), 'Precip_8_12mmhr'] = 1
    TRMM_Rabi[i]['Precip_8_12mmhr']=TRMM_Rabi[i]['Precip_8_12mmhr']*3
    
     #Find hoursless between 12-15mmhr
    TRMM_Rabi[i]['Precip_12_15mmhr'] = 0
    TRMM_Rabi[i].loc[(TRMM_Rabi[i]['Precip'] > 12) & (TRMM_Rabi[i]['Precip'] <15), 'Precip_12_15mmhr'] = 1
    TRMM_Rabi[i]['Precip_12_15mmhr']=TRMM_Rabi[i]['Precip_12_15mmhr']*3
    
    

In [7]:
def dfname(data):
    name =[x for x in globals() if globals()[x] is data][0]
    return name

def ColumnRename(df):
    for i in df:
        stat = dfname(df)
        stat = stat[9:13]
        df[i].columns = [str(col) + stat for col in df[i].columns]

            

In [8]:
#make dictionaries for GLDAS min, max, mean, sum Weekly values
GLDA_Rabi_Min_Weekly = copy.deepcopy(GLDAS_Rabi)
GLDA_Rabi_Ave_Weekly = copy.deepcopy(GLDAS_Rabi)
GLDA_Rabi_Med_Weekly = copy.deepcopy(GLDAS_Rabi)
GLDA_Rabi_Max_Weekly = copy.deepcopy(GLDAS_Rabi)
GLDA_Rabi_Sum_Weekly = copy.deepcopy(GLDAS_Rabi_Sums)

#make dictionaries for GLDAS min, max, mean, sum monthly values
GLDA_Rabi_Min_Monthly = copy.deepcopy(GLDAS_Rabi)
GLDA_Rabi_Ave_Monthly = copy.deepcopy(GLDAS_Rabi)
GLDA_Rabi_Med_Monthly = copy.deepcopy(GLDAS_Rabi)
GLDA_Rabi_Max_Monthly = copy.deepcopy(GLDAS_Rabi)
GLDA_Rabi_Sum_Monthly = copy.deepcopy(GLDAS_Rabi_Sums)

#make dictionaries for GLDAS min, max, mean, sum Season values
GLDA_Rabi_Min_Season = copy.deepcopy(GLDAS_Rabi)
GLDA_Rabi_Ave_Season = copy.deepcopy(GLDAS_Rabi)
GLDA_Rabi_Med_Season = copy.deepcopy(GLDAS_Rabi)
GLDA_Rabi_Max_Season = copy.deepcopy(GLDAS_Rabi)
GLDA_Rabi_Sum_Season = copy.deepcopy(GLDAS_Rabi_Sums)

#make dictionaries for TRMM min, max, mean, sum Weekly values
TRMM_Rabi_Sum_Weekly = copy.deepcopy(TRMM_Rabi)

#make dictionaries for TRMM min, max, mean, sum monthly values
TRMM_Rabi_Sum_Monthly = copy.deepcopy(TRMM_Rabi)

#make dictionaries for TRMM min, max, mean, sum Season values
TRMM_Rabi_Sum_Season = copy.deepcopy(TRMM_Rabi)



for i in GLDAS_Rabi:
    #Weekly
    GLDA_Rabi_Min_Weekly[i] = GLDA_Rabi_Min_Weekly[i].resample('W').min()
    GLDA_Rabi_Ave_Weekly[i] = GLDA_Rabi_Ave_Weekly[i].resample('W').mean()  
    GLDA_Rabi_Med_Weekly[i] = GLDA_Rabi_Med_Weekly[i].resample('W').median()
    GLDA_Rabi_Max_Weekly[i] = GLDA_Rabi_Max_Weekly[i].resample('W').max()
    GLDA_Rabi_Sum_Weekly[i] = GLDA_Rabi_Sum_Weekly[i].resample('W').sum() 
    TRMM_Rabi_Sum_Weekly[i] = TRMM_Rabi_Sum_Weekly[i].resample('W').sum()
   

    #Monthly
    GLDA_Rabi_Min_Monthly[i] = GLDA_Rabi_Min_Monthly[i].resample('M').min()
    GLDA_Rabi_Ave_Monthly[i] = GLDA_Rabi_Ave_Monthly[i].resample('M').mean()
    GLDA_Rabi_Med_Monthly[i] = GLDA_Rabi_Med_Monthly[i].resample('M').median()
    GLDA_Rabi_Max_Monthly[i] = GLDA_Rabi_Max_Monthly[i].resample('M').max()
    GLDA_Rabi_Sum_Monthly[i] = GLDA_Rabi_Sum_Monthly[i].resample('M').sum()
    TRMM_Rabi_Sum_Monthly[i] = TRMM_Rabi_Sum_Monthly[i].resample('M').sum()
    
    #Find the annual values
    GLDA_Rabi_Min_Season[i] = pd.DataFrame(GLDA_Rabi_Min_Season[i].min()).T
    GLDA_Rabi_Ave_Season[i] = pd.DataFrame(GLDA_Rabi_Ave_Season[i].mean()).T
    GLDA_Rabi_Med_Season[i] = pd.DataFrame(GLDA_Rabi_Med_Season[i].median()).T
    GLDA_Rabi_Max_Season[i] = pd.DataFrame(GLDA_Rabi_Max_Season[i].max()).T
    GLDA_Rabi_Sum_Season[i] = pd.DataFrame(GLDA_Rabi_Sum_Season[i].sum()).T
    TRMM_Rabi_Sum_Season[i] = pd.DataFrame(TRMM_Rabi_Sum_Season[i].sum()).T

    
#need to create max month precip
TRMM_Rabi_Max_Monthly = copy.deepcopy(TRMM_Rabi_Sum_Monthly)


for i in GLDAS_Rabi:
    TRMM_Rabi_Max_Monthly[i] = pd.DataFrame(TRMM_Rabi_Max_Monthly[i].max()).T
    
#need to remove features that do not makes sense
GLDAS_remcol = ['Air_temp_C', 'SM_0_10cm', 'SM_10_40cm', 'SM_40_100cm', 'STemp_0_10cm',
       'STemp_10_40cm']
for i in GLDAS_Rabi:
    GLDA_Rabi_Sum_Weekly[i]=GLDA_Rabi_Sum_Weekly[i].drop(columns = GLDAS_remcol)
    GLDA_Rabi_Sum_Monthly[i]=GLDA_Rabi_Sum_Monthly[i].drop(columns = GLDAS_remcol)
    GLDA_Rabi_Sum_Season[i]=GLDA_Rabi_Sum_Season[i].drop(columns = GLDAS_remcol)
    
    
#rename all of the columns in each df
ColumnRename(GLDA_Rabi_Min_Weekly)
ColumnRename(GLDA_Rabi_Ave_Weekly)    
ColumnRename(GLDA_Rabi_Med_Weekly)
ColumnRename(GLDA_Rabi_Max_Weekly)
ColumnRename(GLDA_Rabi_Sum_Weekly)
ColumnRename(TRMM_Rabi_Sum_Weekly)
    
ColumnRename(GLDA_Rabi_Min_Monthly)
ColumnRename(GLDA_Rabi_Ave_Monthly)    
ColumnRename(GLDA_Rabi_Med_Monthly)
ColumnRename(GLDA_Rabi_Max_Monthly)
ColumnRename(GLDA_Rabi_Sum_Monthly)
ColumnRename(TRMM_Rabi_Sum_Monthly)   
    
ColumnRename(GLDA_Rabi_Min_Season)
ColumnRename(GLDA_Rabi_Ave_Season)    
ColumnRename(GLDA_Rabi_Med_Season)
ColumnRename(GLDA_Rabi_Max_Season)
ColumnRename(GLDA_Rabi_Sum_Season)
ColumnRename(TRMM_Rabi_Sum_Season)
      
ColumnRename(TRMM_Rabi_Max_Monthly)
    

In [9]:
#make a weekid function
def WeekID(df):
    for i in df:
        GS_length = len(df[i])+1
        weekrange = np.arange(1,GS_length,1)
        Weeks = []
        for j in weekrange:
            Weeks.append('Week_'+str(j))
            
        df[i]['TimeStep'] = Weeks
        
#need to flatten monthly sums DF's to make monthly features
def flatten(df):
    for i in df:
        time = df[i]['TimeStep'].copy()
        del df[i]['TimeStep']
        columns = list(df[i].columns.copy())
        updated_columns = list()
        
        for j in time:
            for k in columns:
                updated_columns.append(j+ '_' + k)
       
        Flat_Array = np.array(df[i].copy())
        df_len = Flat_Array.shape[0]*Flat_Array.shape[1]
        Flat_Array = np.array(Flat_Array).reshape(1,df_len)
        df[i] = pd.DataFrame(Flat_Array, columns = updated_columns)
        
        #put year into DF for reference and to the front
        df[i]['Year']= i[-4:]
        df[i]=df[i].set_index('Year')
        
        

def DistrictMetrics(District_Years, MinW, AveW, MedW, MaxW, SumW, SumPW, 
                    MinM, AveM, MedM, MaxM, SumM, SumPM,
                    MinS, AveS, MedS, MaxS, SumS, SumPS,MaxMS):
    #create a district specific dictionary
    MinW = {key: MinW[key] for key in District_Years}
    AveW = {key: AveW[key] for key in District_Years}
    MedW = {key: MedW[key] for key in District_Years}
    MaxW = {key: MaxW[key] for key in District_Years}
    SumW = {key: SumW[key] for key in District_Years}
    SumPW = {key: SumPW[key] for key in District_Years}
    
    MinM = {key: MinM[key] for key in District_Years}
    AveM = {key: AveM[key] for key in District_Years}
    MedM = {key: MedM[key] for key in District_Years}
    MaxM = {key: MaxM[key] for key in District_Years}
    SumM = {key: SumM[key] for key in District_Years}
    SumPM = {key: SumPM[key] for key in District_Years}
    
    MinS = {key: MinS[key] for key in District_Years}
    AveS = {key: AveS[key] for key in District_Years}
    MedS = {key: MedS[key] for key in District_Years}
    MaxS = {key: MaxS[key] for key in District_Years}
    SumS = {key: SumS[key] for key in District_Years}
    SumPS = {key: SumPS[key] for key in District_Years}
    MaxMS = {key: MaxMS[key] for key in District_Years}
    
    #Make new DF's to hold the metrics
    WeeklyMin = pd.DataFrame()
    WeeklyAve = pd.DataFrame()
    WeeklyMed = pd.DataFrame()
    WeeklyMax = pd.DataFrame()
    WeeklySum = pd.DataFrame()
    WeeklySumP = pd.DataFrame()
    
    MonthlyMin = pd.DataFrame()
    MonthlyAve = pd.DataFrame()
    MonthlyMed = pd.DataFrame()
    MonthlyMax = pd.DataFrame()
    MonthlySum = pd.DataFrame()
    MonthlySumP = pd.DataFrame()
    
    SeasonMin = pd.DataFrame()
    SeasonAve = pd.DataFrame()
    SeasonMed = pd.DataFrame()
    SeasonMax = pd.DataFrame()
    SeasonSum = pd.DataFrame()
    SeasonSumP = pd.DataFrame()
    SeasonMaxMS = pd.DataFrame()
    
    
    for i in MinW:  
        WeeklyMin = WeeklyMin.append(MinW[i])
        WeeklyAve = WeeklyAve.append(AveW[i])
        WeeklyMed = WeeklyMed.append(MedW[i])
        WeeklyMax = WeeklyMax.append(MaxW[i])
        WeeklySum = WeeklySum.append(SumW[i])
        WeeklySumP = WeeklySumP.append(SumPW[i])
   
    
    for i in MinM:  
        MonthlyMin = MonthlyMin.append(MinM[i])
        MonthlyAve = MonthlyAve.append(AveM[i])
        MonthlyMed = MonthlyMed.append(MedM[i])
        MonthlyMax = MonthlyMax.append(MaxM[i])
        MonthlySum = MonthlySum.append(SumM[i])
        MonthlySumP = MonthlySumP.append(SumPM[i])
        
    for i in MinS:  
        SeasonMin = SeasonMin.append(MinS[i])
        SeasonAve = SeasonAve.append(AveS[i])
        SeasonMed = SeasonMed.append(MedS[i])
        SeasonMax = SeasonMax.append(MaxS[i])
        SeasonSum = SeasonSum.append(SumS[i])
        SeasonSumP = SeasonSumP.append(SumPS[i])
        SeasonMaxMS = SeasonMaxMS.append(MaxMS[i])
        
    Dist_DF = pd.concat([WeeklyMin,
                         WeeklyAve.sort_index(),
                         WeeklyMed.sort_index(),
                         WeeklyMax.sort_index(),
                         WeeklySum.sort_index(),
                         WeeklySumP.sort_index(),
                         
                         MonthlyMin.sort_index(),
                         MonthlyAve.sort_index(),
                         MonthlyMed.sort_index(),
                         MonthlyMax.sort_index(),
                         MonthlySum.sort_index(),
                         MonthlySumP.sort_index(),
                         
                         SeasonMin.sort_index(),
                         SeasonAve.sort_index(),
                         SeasonMed.sort_index(),
                         SeasonMax.sort_index(),
                         SeasonSum.sort_index(),
                         SeasonSumP.sort_index(),
                        SeasonMaxMS.sort_index()], axis = 1)
    Dist_DF = pd.DataFrame(Dist_DF)
    Dist_DF = Dist_DF.sort_index()

    return Dist_DF

In [10]:
#make WeekId's
WeekID(GLDA_Rabi_Min_Weekly)
WeekID(GLDA_Rabi_Ave_Weekly)
WeekID(GLDA_Rabi_Med_Weekly)
WeekID(GLDA_Rabi_Max_Weekly)
WeekID(GLDA_Rabi_Sum_Weekly)
WeekID(TRMM_Rabi_Sum_Weekly)

months = ['Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr']

#Give all of the months/Years a timeID
for i in TRMM_Rabi:
    GLDA_Rabi_Min_Monthly[i]['TimeStep'] = months
    GLDA_Rabi_Ave_Monthly[i]['TimeStep'] = months
    GLDA_Rabi_Med_Monthly[i]['TimeStep'] = months
    GLDA_Rabi_Max_Monthly[i]['TimeStep'] = months
    GLDA_Rabi_Sum_Monthly[i]['TimeStep'] = months
    TRMM_Rabi_Sum_Monthly[i]['TimeStep'] = months
    
    GLDA_Rabi_Min_Season[i]['TimeStep'] = 'Season'
    GLDA_Rabi_Ave_Season[i]['TimeStep'] = 'Season'
    GLDA_Rabi_Med_Season[i]['TimeStep'] = 'Season'
    GLDA_Rabi_Max_Season[i]['TimeStep'] = 'Season'
    GLDA_Rabi_Sum_Season[i]['TimeStep'] = 'Season'
    TRMM_Rabi_Sum_Season[i]['TimeStep'] = 'Season'
    TRMM_Rabi_Max_Monthly[i]['TimeStep'] = 'Season'
    
#flatten all of the DataFrames
#Weekly
flatten(GLDA_Rabi_Min_Weekly)
flatten(GLDA_Rabi_Ave_Weekly)
flatten(GLDA_Rabi_Med_Weekly)
flatten(GLDA_Rabi_Max_Weekly)
flatten(GLDA_Rabi_Sum_Weekly)
flatten(TRMM_Rabi_Sum_Weekly)

#Monthly
flatten(GLDA_Rabi_Min_Monthly)
flatten(GLDA_Rabi_Ave_Monthly)
flatten(GLDA_Rabi_Med_Monthly)
flatten(GLDA_Rabi_Max_Monthly)
flatten(GLDA_Rabi_Sum_Monthly)
flatten(TRMM_Rabi_Sum_Monthly)

#Season
flatten(GLDA_Rabi_Min_Season)
flatten(GLDA_Rabi_Ave_Season)
flatten(GLDA_Rabi_Med_Season)
flatten(GLDA_Rabi_Max_Season)
flatten(GLDA_Rabi_Sum_Season)
flatten(TRMM_Rabi_Sum_Season)
flatten(TRMM_Rabi_Max_Monthly)
   


In [11]:
#separate data by district
AttG = [k for k in GLDAS_Rabi.keys() if k.startswith('Att')]
ChaG = [k for k in GLDAS_Rabi.keys() if k.startswith('Cha')]
JheG = [k for k in GLDAS_Rabi.keys() if k.startswith('Jhe')]
RawG = [k for k in GLDAS_Rabi.keys() if k.startswith('Raw')]

#use DistrictMetrics function to put into district dataframes
Attock = DistrictMetrics(AttG,
            GLDA_Rabi_Min_Weekly, GLDA_Rabi_Ave_Weekly, GLDA_Rabi_Med_Weekly, GLDA_Rabi_Max_Weekly, GLDA_Rabi_Sum_Weekly,TRMM_Rabi_Sum_Weekly,
            GLDA_Rabi_Min_Monthly, GLDA_Rabi_Ave_Monthly, GLDA_Rabi_Med_Monthly, GLDA_Rabi_Max_Monthly, GLDA_Rabi_Sum_Monthly,TRMM_Rabi_Sum_Monthly,
            GLDA_Rabi_Min_Season, GLDA_Rabi_Ave_Season, GLDA_Rabi_Med_Season, GLDA_Rabi_Max_Season, GLDA_Rabi_Sum_Season,TRMM_Rabi_Sum_Season,
                        TRMM_Rabi_Max_Monthly)

Chakwal = DistrictMetrics(ChaG,
            GLDA_Rabi_Min_Weekly, GLDA_Rabi_Ave_Weekly, GLDA_Rabi_Med_Weekly, GLDA_Rabi_Max_Weekly, GLDA_Rabi_Sum_Weekly,TRMM_Rabi_Sum_Weekly,
            GLDA_Rabi_Min_Monthly, GLDA_Rabi_Ave_Monthly, GLDA_Rabi_Med_Monthly, GLDA_Rabi_Max_Monthly, GLDA_Rabi_Sum_Monthly,TRMM_Rabi_Sum_Monthly,
            GLDA_Rabi_Min_Season, GLDA_Rabi_Ave_Season, GLDA_Rabi_Med_Season, GLDA_Rabi_Max_Season, GLDA_Rabi_Sum_Season,TRMM_Rabi_Sum_Season,
                        TRMM_Rabi_Max_Monthly)

Jhelum = DistrictMetrics(JheG,
            GLDA_Rabi_Min_Weekly, GLDA_Rabi_Ave_Weekly, GLDA_Rabi_Med_Weekly, GLDA_Rabi_Max_Weekly, GLDA_Rabi_Sum_Weekly,TRMM_Rabi_Sum_Weekly,
            GLDA_Rabi_Min_Monthly, GLDA_Rabi_Ave_Monthly, GLDA_Rabi_Med_Monthly, GLDA_Rabi_Max_Monthly, GLDA_Rabi_Sum_Monthly,TRMM_Rabi_Sum_Monthly,
            GLDA_Rabi_Min_Season, GLDA_Rabi_Ave_Season, GLDA_Rabi_Med_Season, GLDA_Rabi_Max_Season, GLDA_Rabi_Sum_Season,TRMM_Rabi_Sum_Season,
                        TRMM_Rabi_Max_Monthly)

Rawalpindi = DistrictMetrics(RawG,
            GLDA_Rabi_Min_Weekly, GLDA_Rabi_Ave_Weekly, GLDA_Rabi_Med_Weekly, GLDA_Rabi_Max_Weekly, GLDA_Rabi_Sum_Weekly,TRMM_Rabi_Sum_Weekly,
            GLDA_Rabi_Min_Monthly, GLDA_Rabi_Ave_Monthly, GLDA_Rabi_Med_Monthly, GLDA_Rabi_Max_Monthly, GLDA_Rabi_Sum_Monthly,TRMM_Rabi_Sum_Monthly,
            GLDA_Rabi_Min_Season, GLDA_Rabi_Ave_Season, GLDA_Rabi_Med_Season, GLDA_Rabi_Max_Season, GLDA_Rabi_Sum_Season,TRMM_Rabi_Sum_Season,
                        TRMM_Rabi_Max_Monthly)


#Add a District column for reference
Attock['District'] = 'Att'
Chakwal['District'] = 'Cha'
Jhelum['District'] = 'Jhe'
Rawalpindi['District'] = 'Raw'

#Add Wheat Yield
Attock['Wheat_ton_ha'] = list(Wheat['Att_ton_ha_W'])
Chakwal['Wheat_ton_ha'] = list(Wheat['Cha_ton_ha_W'])
Jhelum['Wheat_ton_ha'] = list(Wheat['Jhe_ton_ha_W'])
Rawalpindi['Wheat_ton_ha'] = list(Wheat['Raw_ton_ha_W'])

#drop 2010 as no wheat data
Attock = Attock.drop(['2010'])
Chakwal=Chakwal.drop(['2010'])
Jhelum=Jhelum.drop(['2010'])
Rawalpindi=Rawalpindi.drop(['2010'])



In [12]:
#now lets combine the DF
Potohar = pd.DataFrame()
Potohar = Potohar.append([Attock, Chakwal, Jhelum, Rawalpindi])

#Just add a few more combined monthly features
#accumulated precipitation
Potohar['Nov_Dec_Summonthly_Precip_Sum']= Potohar['Nov_Precip_Sum']+ Potohar['Dec_Precip_Sum']
Potohar['Nov_Jan_Summonthly_Precip_Sum']= Potohar['Nov_Precip_Sum']+ Potohar['Dec_Precip_Sum']+Potohar['Jan_Precip_Sum'] 
Potohar['Nov_Feb_Summonthly_Precip_Sum']= Potohar['Nov_Precip_Sum']+ Potohar['Dec_Precip_Sum']+Potohar['Jan_Precip_Sum'] + Potohar['Feb_Precip_Sum']
Potohar['Nov_Feb_Summonthly_Precip_Sum']= Potohar['Nov_Precip_Sum']+ Potohar['Dec_Precip_Sum']+Potohar['Jan_Precip_Sum'] + Potohar['Feb_Precip_Sum']+Potohar['Mar_Precip_Sum']
Potohar['Season_Precip_Sum_mm']= Potohar['Nov_Precip_Sum']+ Potohar['Dec_Precip_Sum']+Potohar['Jan_Precip_Sum'] + Potohar['Feb_Precip_Sum']+Potohar['Mar_Precip_Sum']+Potohar['Apr_Precip_Sum']

Potohar['Jan_Feb_Summonthly_Precip_Sum']= Potohar['Jan_Precip_Sum'] + Potohar['Feb_Precip_Sum']
Potohar['Jan_Mar_Summonthly_Precip_Sum']= Potohar['Jan_Precip_Sum'] + Potohar['Feb_Precip_Sum']+ Potohar['Mar_Precip_Sum']

Potohar['Feb_Mar_Summonthly_Precip_Sum']= Potohar['Feb_Precip_Sum']+ Potohar['Mar_Precip_Sum']

#aggregated precipitation hours
Potohar['Nov_Dec_Summonthly_PrecipHrs_Sum']= Potohar['Nov_PrecipHrs_Sum']+ Potohar['Dec_PrecipHrs_Sum']
Potohar['Nov_Jan_Summonthly_PrecipHrs_Sum']= Potohar['Nov_PrecipHrs_Sum']+ Potohar['Dec_PrecipHrs_Sum']+Potohar['Jan_PrecipHrs_Sum'] 
Potohar['Nov_Feb_Summonthly_PrecipHrs_Sum']= Potohar['Nov_PrecipHrs_Sum']+ Potohar['Dec_PrecipHrs_Sum']+Potohar['Jan_PrecipHrs_Sum'] + Potohar['Feb_PrecipHrs_Sum']
Potohar['Nov_Feb_Summonthly_PrecipHrs_Sum']= Potohar['Nov_PrecipHrs_Sum']+ Potohar['Dec_PrecipHrs_Sum']+Potohar['Jan_PrecipHrs_Sum'] + Potohar['Feb_PrecipHrs_Sum']+Potohar['Mar_PrecipHrs_Sum']
Potohar['Season_PrecipHrs_Sum_mm']= Potohar['Nov_PrecipHrs_Sum']+ Potohar['Dec_PrecipHrs_Sum']+Potohar['Jan_PrecipHrs_Sum'] + Potohar['Feb_PrecipHrs_Sum']+Potohar['Mar_PrecipHrs_Sum']+Potohar['Apr_PrecipHrs_Sum']

Potohar['Jan_Feb_Summonthly_PrecipHrs_Sum']= Potohar['Jan_PrecipHrs_Sum'] + Potohar['Feb_PrecipHrs_Sum']
Potohar['Jan_Mar_Summonthly_PrecipHrs_Sum']= Potohar['Jan_PrecipHrs_Sum'] + Potohar['Feb_PrecipHrs_Sum']+ Potohar['Mar_PrecipHrs_Sum']

Potohar['Feb_Mar_Summonthly_PrecipHrs_Sum']= Potohar['Feb_PrecipHrs_Sum']+ Potohar['Mar_PrecipHrs_Sum']

#aggregated precipitation intensity hours 1-5mmhr
Potohar['Nov_Dec_Summonthly_Precip_1_5mmhr_Sum']= Potohar['Nov_Precip_1_5mmhr_Sum']+ Potohar['Dec_Precip_1_5mmhr_Sum']
Potohar['Nov_Jan_Summonthly_Precip_1_5mmhr_Sum']= Potohar['Nov_Precip_1_5mmhr_Sum']+ Potohar['Dec_Precip_1_5mmhr_Sum']+Potohar['Jan_Precip_1_5mmhr_Sum'] 
Potohar['Nov_Feb_Summonthly_Precip_1_5mmhr_Sum']= Potohar['Nov_Precip_1_5mmhr_Sum']+ Potohar['Dec_Precip_1_5mmhr_Sum']+Potohar['Jan_Precip_1_5mmhr_Sum'] + Potohar['Feb_Precip_1_5mmhr_Sum']
Potohar['Nov_Feb_Summonthly_Precip_1_5mmhr_Sum']= Potohar['Nov_Precip_1_5mmhr_Sum']+ Potohar['Dec_Precip_1_5mmhr_Sum']+Potohar['Jan_Precip_1_5mmhr_Sum'] + Potohar['Feb_Precip_1_5mmhr_Sum']+Potohar['Mar_Precip_1_5mmhr_Sum']
Potohar['Season_Precip_1_5mmhr_Sum_mm']= Potohar['Nov_Precip_1_5mmhr_Sum']+ Potohar['Dec_Precip_1_5mmhr_Sum']+Potohar['Jan_Precip_1_5mmhr_Sum'] + Potohar['Feb_Precip_1_5mmhr_Sum']+Potohar['Mar_Precip_1_5mmhr_Sum']+Potohar['Apr_Precip_1_5mmhr_Sum']

Potohar['Jan_Feb_Summonthly_Precip_1_5mmhr_Sum']= Potohar['Jan_Precip_1_5mmhr_Sum'] + Potohar['Feb_Precip_1_5mmhr_Sum']
Potohar['Jan_Mar_Summonthly_Precip_1_5mmhr_Sum']= Potohar['Jan_Precip_1_5mmhr_Sum'] + Potohar['Feb_Precip_1_5mmhr_Sum']+ Potohar['Mar_Precip_1_5mmhr_Sum']

Potohar['Feb_Mar_Summonthly_Precip_1_5mmhr_Sum']= Potohar['Feb_Precip_1_5mmhr_Sum']+ Potohar['Mar_Precip_1_5mmhr_Sum']

#aggregated precipitation intensity hours 5-8mmhr
Potohar['Nov_Dec_Summonthly_Precip_5_8mmhr_Sum']= Potohar['Nov_Precip_5_8mmhr_Sum']+ Potohar['Dec_Precip_5_8mmhr_Sum']
Potohar['Nov_Jan_Summonthly_Precip_5_8mmhr_Sum']= Potohar['Nov_Precip_5_8mmhr_Sum']+ Potohar['Dec_Precip_5_8mmhr_Sum']+Potohar['Jan_Precip_5_8mmhr_Sum'] 
Potohar['Nov_Feb_Summonthly_Precip_5_8mmhr_Sum']= Potohar['Nov_Precip_5_8mmhr_Sum']+ Potohar['Dec_Precip_5_8mmhr_Sum']+Potohar['Jan_Precip_5_8mmhr_Sum'] + Potohar['Feb_Precip_5_8mmhr_Sum']
Potohar['Nov_Feb_Summonthly_Precip_5_8mmhr_Sum']= Potohar['Nov_Precip_5_8mmhr_Sum']+ Potohar['Dec_Precip_5_8mmhr_Sum']+Potohar['Jan_Precip_5_8mmhr_Sum'] + Potohar['Feb_Precip_5_8mmhr_Sum']+Potohar['Mar_Precip_5_8mmhr_Sum']
Potohar['Season_Precip_5_8mmhr_Sum_mm']= Potohar['Nov_Precip_5_8mmhr_Sum']+ Potohar['Dec_Precip_5_8mmhr_Sum']+Potohar['Jan_Precip_5_8mmhr_Sum'] + Potohar['Feb_Precip_5_8mmhr_Sum']+Potohar['Mar_Precip_5_8mmhr_Sum']+Potohar['Apr_Precip_5_8mmhr_Sum']

Potohar['Jan_Feb_Summonthly_Precip_5_8mmhr_Sum']= Potohar['Jan_Precip_5_8mmhr_Sum'] + Potohar['Feb_Precip_5_8mmhr_Sum']
Potohar['Jan_Mar_Summonthly_Precip_5_8mmhr_Sum']= Potohar['Jan_Precip_5_8mmhr_Sum'] + Potohar['Feb_Precip_5_8mmhr_Sum']+ Potohar['Mar_Precip_5_8mmhr_Sum']

Potohar['Feb_Mar_Summonthly_Precip_5_8mmhr_Sum']= Potohar['Feb_Precip_5_8mmhr_Sum']+ Potohar['Mar_Precip_5_8mmhr_Sum']


#aggregated precipitation intensity hours 8-12mmhr
Potohar['Nov_Dec_Summonthly_Precip_8_12mmhr_Sum']= Potohar['Nov_Precip_8_12mmhr_Sum']+ Potohar['Dec_Precip_8_12mmhr_Sum']
Potohar['Nov_Jan_Summonthly_Precip_8_12mmhr_Sum']= Potohar['Nov_Precip_8_12mmhr_Sum']+ Potohar['Dec_Precip_8_12mmhr_Sum']+Potohar['Jan_Precip_8_12mmhr_Sum'] 
Potohar['Nov_Feb_Summonthly_Precip_8_12mmhr_Sum']= Potohar['Nov_Precip_8_12mmhr_Sum']+ Potohar['Dec_Precip_8_12mmhr_Sum']+Potohar['Jan_Precip_8_12mmhr_Sum'] + Potohar['Feb_Precip_8_12mmhr_Sum']
Potohar['Nov_Feb_Summonthly_Precip_8_12mmhr_Sum']= Potohar['Nov_Precip_8_12mmhr_Sum']+ Potohar['Dec_Precip_8_12mmhr_Sum']+Potohar['Jan_Precip_8_12mmhr_Sum'] + Potohar['Feb_Precip_8_12mmhr_Sum']+Potohar['Mar_Precip_8_12mmhr_Sum']
Potohar['Season_Precip_8_12mmhr_Sum_mm']= Potohar['Nov_Precip_8_12mmhr_Sum']+ Potohar['Dec_Precip_8_12mmhr_Sum']+Potohar['Jan_Precip_8_12mmhr_Sum'] + Potohar['Feb_Precip_8_12mmhr_Sum']+Potohar['Mar_Precip_8_12mmhr_Sum']+Potohar['Apr_Precip_8_12mmhr_Sum']

Potohar['Jan_Feb_Summonthly_Precip_8_12mmhr_Sum']= Potohar['Jan_Precip_8_12mmhr_Sum'] + Potohar['Feb_Precip_8_12mmhr_Sum']
Potohar['Jan_Mar_Summonthly_Precip_8_12mmhr_Sum']= Potohar['Jan_Precip_8_12mmhr_Sum'] + Potohar['Feb_Precip_8_12mmhr_Sum']+ Potohar['Mar_Precip_8_12mmhr_Sum']

Potohar['Feb_Mar_Summonthly_Precip_8_12mmhr_Sum']= Potohar['Feb_Precip_8_12mmhr_Sum']+ Potohar['Mar_Precip_8_12mmhr_Sum']


#aggregated precipitation intensity hours 12-15mmhr
Potohar['Nov_Dec_Summonthly_Precip_12_15mmhr_Sum']= Potohar['Nov_Precip_12_15mmhr_Sum']+ Potohar['Dec_Precip_12_15mmhr_Sum']
Potohar['Nov_Jan_Summonthly_Precip_12_15mmhr_Sum']= Potohar['Nov_Precip_12_15mmhr_Sum']+ Potohar['Dec_Precip_12_15mmhr_Sum']+Potohar['Jan_Precip_12_15mmhr_Sum'] 
Potohar['Nov_Feb_Summonthly_Precip_12_15mmhr_Sum']= Potohar['Nov_Precip_12_15mmhr_Sum']+ Potohar['Dec_Precip_12_15mmhr_Sum']+Potohar['Jan_Precip_12_15mmhr_Sum'] + Potohar['Feb_Precip_12_15mmhr_Sum']
Potohar['Nov_Feb_Summonthly_Precip_12_15mmhr_Sum']= Potohar['Nov_Precip_12_15mmhr_Sum']+ Potohar['Dec_Precip_12_15mmhr_Sum']+Potohar['Jan_Precip_12_15mmhr_Sum'] + Potohar['Feb_Precip_12_15mmhr_Sum']+Potohar['Mar_Precip_12_15mmhr_Sum']
Potohar['Season_Precip_12_15mmhr_Sum_mm']= Potohar['Nov_Precip_12_15mmhr_Sum']+ Potohar['Dec_Precip_12_15mmhr_Sum']+Potohar['Jan_Precip_12_15mmhr_Sum'] + Potohar['Feb_Precip_12_15mmhr_Sum']+Potohar['Mar_Precip_12_15mmhr_Sum']+Potohar['Apr_Precip_12_15mmhr_Sum']

Potohar['Jan_Feb_Summonthly_Precip_12_15mmhr_Sum']= Potohar['Jan_Precip_12_15mmhr_Sum'] + Potohar['Feb_Precip_12_15mmhr_Sum']
Potohar['Jan_Mar_Summonthly_Precip_12_15mmhr_Sum']= Potohar['Jan_Precip_12_15mmhr_Sum'] + Potohar['Feb_Precip_12_15mmhr_Sum']+ Potohar['Mar_Precip_12_15mmhr_Sum']

Potohar['Feb_Mar_Summonthly_Precip_12_15mmhr_Sum']= Potohar['Feb_Precip_12_15mmhr_Sum']+ Potohar['Mar_Precip_12_15mmhr_Sum']



#aggregated precipitation intensity hours above 2mmhr
Potohar['Nov_Dec_Summonthly_Precip_above2mmhr_Sum']= Potohar['Nov_Precip_above2mmhr_Sum']+ Potohar['Dec_Precip_above2mmhr_Sum']
Potohar['Nov_Jan_Summonthly_Precip_above2mmhr_Sum']= Potohar['Nov_Precip_above2mmhr_Sum']+ Potohar['Dec_Precip_above2mmhr_Sum']+Potohar['Jan_Precip_above2mmhr_Sum'] 
Potohar['Nov_Feb_Summonthly_Precip_above2mmhr_Sum']= Potohar['Nov_Precip_above2mmhr_Sum']+ Potohar['Dec_Precip_above2mmhr_Sum']+Potohar['Jan_Precip_above2mmhr_Sum'] + Potohar['Feb_Precip_above2mmhr_Sum']
Potohar['Nov_Feb_Summonthly_Precip_above2mmhr_Sum']= Potohar['Nov_Precip_above2mmhr_Sum']+ Potohar['Dec_Precip_above2mmhr_Sum']+Potohar['Jan_Precip_above2mmhr_Sum'] + Potohar['Feb_Precip_above2mmhr_Sum']+Potohar['Mar_Precip_above2mmhr_Sum']
Potohar['Season_Precip_above2mmhr_Sum_mm']= Potohar['Nov_Precip_above2mmhr_Sum']+ Potohar['Dec_Precip_above2mmhr_Sum']+Potohar['Jan_Precip_above2mmhr_Sum'] + Potohar['Feb_Precip_above2mmhr_Sum']+Potohar['Mar_Precip_above2mmhr_Sum']+Potohar['Apr_Precip_above2mmhr_Sum']

Potohar['Jan_Feb_Summonthly_Precip_above2mmhr_Sum']= Potohar['Jan_Precip_above2mmhr_Sum'] + Potohar['Feb_Precip_above2mmhr_Sum']
Potohar['Jan_Mar_Summonthly_Precip_above2mmhr_Sum']= Potohar['Jan_Precip_above2mmhr_Sum'] + Potohar['Feb_Precip_above2mmhr_Sum']+ Potohar['Mar_Precip_above2mmhr_Sum']

Potohar['Feb_Mar_Summonthly_Precip_above2mmhr_Sum']= Potohar['Feb_Precip_above2mmhr_Sum']+ Potohar['Mar_Precip_above2mmhr_Sum']

#aggregated precipitation intensity hours above 5mmhr
Potohar['Nov_Dec_Summonthly_Precip_above5mmhr_Sum']= Potohar['Nov_Precip_above5mmhr_Sum']+ Potohar['Dec_Precip_above5mmhr_Sum']
Potohar['Nov_Jan_Summonthly_Precip_above5mmhr_Sum']= Potohar['Nov_Precip_above5mmhr_Sum']+ Potohar['Dec_Precip_above5mmhr_Sum']+Potohar['Jan_Precip_above5mmhr_Sum'] 
Potohar['Nov_Feb_Summonthly_Precip_above5mmhr_Sum']= Potohar['Nov_Precip_above5mmhr_Sum']+ Potohar['Dec_Precip_above5mmhr_Sum']+Potohar['Jan_Precip_above5mmhr_Sum'] + Potohar['Feb_Precip_above5mmhr_Sum']
Potohar['Nov_Feb_Summonthly_Precip_above5mmhr_Sum']= Potohar['Nov_Precip_above5mmhr_Sum']+ Potohar['Dec_Precip_above5mmhr_Sum']+Potohar['Jan_Precip_above5mmhr_Sum'] + Potohar['Feb_Precip_above5mmhr_Sum']+Potohar['Mar_Precip_above5mmhr_Sum']
Potohar['Season_Precip_above5mmhr_Sum_mm']= Potohar['Nov_Precip_above5mmhr_Sum']+ Potohar['Dec_Precip_above5mmhr_Sum']+Potohar['Jan_Precip_above5mmhr_Sum'] + Potohar['Feb_Precip_above5mmhr_Sum']+Potohar['Mar_Precip_above5mmhr_Sum']+Potohar['Apr_Precip_above5mmhr_Sum']

Potohar['Jan_Feb_Summonthly_Precip_above5mmhr_Sum']= Potohar['Jan_Precip_above5mmhr_Sum'] + Potohar['Feb_Precip_above5mmhr_Sum']
Potohar['Jan_Mar_Summonthly_Precip_above5mmhr_Sum']= Potohar['Jan_Precip_above5mmhr_Sum'] + Potohar['Feb_Precip_above5mmhr_Sum']+ Potohar['Mar_Precip_above5mmhr_Sum']

Potohar['Feb_Mar_Summonthly_Precip_above5mmhr_Sum']= Potohar['Feb_Precip_above5mmhr_Sum']+ Potohar['Mar_Precip_above5mmhr_Sum']


#aggregated precipitation intensity hours above 8mmhr
Potohar['Nov_Dec_Summonthly_Precip_above8mmhr_Sum']= Potohar['Nov_Precip_above8mmhr_Sum']+ Potohar['Dec_Precip_above8mmhr_Sum']
Potohar['Nov_Jan_Summonthly_Precip_above8mmhr_Sum']= Potohar['Nov_Precip_above8mmhr_Sum']+ Potohar['Dec_Precip_above8mmhr_Sum']+Potohar['Jan_Precip_above8mmhr_Sum'] 
Potohar['Nov_Feb_Summonthly_Precip_above8mmhr_Sum']= Potohar['Nov_Precip_above8mmhr_Sum']+ Potohar['Dec_Precip_above8mmhr_Sum']+Potohar['Jan_Precip_above8mmhr_Sum'] + Potohar['Feb_Precip_above8mmhr_Sum']
Potohar['Nov_Feb_Summonthly_Precip_above8mmhr_Sum']= Potohar['Nov_Precip_above8mmhr_Sum']+ Potohar['Dec_Precip_above8mmhr_Sum']+Potohar['Jan_Precip_above8mmhr_Sum'] + Potohar['Feb_Precip_above8mmhr_Sum']+Potohar['Mar_Precip_above8mmhr_Sum']
Potohar['Season_Precip_above8mmhr_Sum_mm']= Potohar['Nov_Precip_above8mmhr_Sum']+ Potohar['Dec_Precip_above8mmhr_Sum']+Potohar['Jan_Precip_above8mmhr_Sum'] + Potohar['Feb_Precip_above8mmhr_Sum']+Potohar['Mar_Precip_above8mmhr_Sum']+Potohar['Apr_Precip_above8mmhr_Sum']

Potohar['Jan_Feb_Summonthly_Precip_above8mmhr_Sum']= Potohar['Jan_Precip_above8mmhr_Sum'] + Potohar['Feb_Precip_above8mmhr_Sum']
Potohar['Jan_Mar_Summonthly_Precip_above8mmhr_Sum']= Potohar['Jan_Precip_above8mmhr_Sum'] + Potohar['Feb_Precip_above8mmhr_Sum']+ Potohar['Mar_Precip_above8mmhr_Sum']

Potohar['Feb_Mar_Summonthly_Precip_above8mmhr_Sum']= Potohar['Feb_Precip_above8mmhr_Sum']+ Potohar['Mar_Precip_above8mmhr_Sum']


#aggregated precipitation intensity hours above 12mmhr
Potohar['Nov_Dec_Summonthly_Precip_above12mmhr_Sum']= Potohar['Nov_Precip_above12mmhr_Sum']+ Potohar['Dec_Precip_above12mmhr_Sum']
Potohar['Nov_Jan_Summonthly_Precip_above12mmhr_Sum']= Potohar['Nov_Precip_above12mmhr_Sum']+ Potohar['Dec_Precip_above12mmhr_Sum']+Potohar['Jan_Precip_above12mmhr_Sum'] 
Potohar['Nov_Feb_Summonthly_Precip_above12mmhr_Sum']= Potohar['Nov_Precip_above12mmhr_Sum']+ Potohar['Dec_Precip_above12mmhr_Sum']+Potohar['Jan_Precip_above12mmhr_Sum'] + Potohar['Feb_Precip_above12mmhr_Sum']
Potohar['Nov_Feb_Summonthly_Precip_above12mmhr_Sum']= Potohar['Nov_Precip_above12mmhr_Sum']+ Potohar['Dec_Precip_above12mmhr_Sum']+Potohar['Jan_Precip_above12mmhr_Sum'] + Potohar['Feb_Precip_above12mmhr_Sum']+Potohar['Mar_Precip_above12mmhr_Sum']
Potohar['Season_Precip_above12mmhr_Sum_mm']= Potohar['Nov_Precip_above12mmhr_Sum']+ Potohar['Dec_Precip_above12mmhr_Sum']+Potohar['Jan_Precip_above12mmhr_Sum'] + Potohar['Feb_Precip_above12mmhr_Sum']+Potohar['Mar_Precip_above12mmhr_Sum']+Potohar['Apr_Precip_above12mmhr_Sum']

Potohar['Jan_Feb_Summonthly_Precip_above12mmhr_Sum']= Potohar['Jan_Precip_above12mmhr_Sum'] + Potohar['Feb_Precip_above12mmhr_Sum']
Potohar['Jan_Mar_Summonthly_Precip_above12mmhr_Sum']= Potohar['Jan_Precip_above12mmhr_Sum'] + Potohar['Feb_Precip_above12mmhr_Sum']+ Potohar['Mar_Precip_above12mmhr_Sum']

Potohar['Feb_Mar_Summonthly_Precip_above12mmhr_Sum']= Potohar['Feb_Precip_above12mmhr_Sum']+ Potohar['Mar_Precip_above12mmhr_Sum']


#aggregated precipitation intensity hours above 15mmhr
Potohar['Nov_Dec_Summonthly_Precip_above15mmhr_Sum']= Potohar['Nov_Precip_above15mmhr_Sum']+ Potohar['Dec_Precip_above15mmhr_Sum']
Potohar['Nov_Jan_Summonthly_Precip_above15mmhr_Sum']= Potohar['Nov_Precip_above15mmhr_Sum']+ Potohar['Dec_Precip_above15mmhr_Sum']+Potohar['Jan_Precip_above15mmhr_Sum'] 
Potohar['Nov_Feb_Summonthly_Precip_above15mmhr_Sum']= Potohar['Nov_Precip_above15mmhr_Sum']+ Potohar['Dec_Precip_above15mmhr_Sum']+Potohar['Jan_Precip_above15mmhr_Sum'] + Potohar['Feb_Precip_above15mmhr_Sum']
Potohar['Nov_Feb_Summonthly_Precip_above15mmhr_Sum']= Potohar['Nov_Precip_above15mmhr_Sum']+ Potohar['Dec_Precip_above15mmhr_Sum']+Potohar['Jan_Precip_above15mmhr_Sum'] + Potohar['Feb_Precip_above15mmhr_Sum']+Potohar['Mar_Precip_above15mmhr_Sum']
Potohar['Season_Precip_above15mmhr_Sum_mm']= Potohar['Nov_Precip_above15mmhr_Sum']+ Potohar['Dec_Precip_above15mmhr_Sum']+Potohar['Jan_Precip_above15mmhr_Sum'] + Potohar['Feb_Precip_above15mmhr_Sum']+Potohar['Mar_Precip_above15mmhr_Sum']+Potohar['Apr_Precip_above15mmhr_Sum']

Potohar['Jan_Feb_Summonthly_Precip_above15mmhr_Sum']= Potohar['Jan_Precip_above15mmhr_Sum'] + Potohar['Feb_Precip_above15mmhr_Sum']
Potohar['Jan_Mar_Summonthly_Precip_above15mmhr_Sum']= Potohar['Jan_Precip_above15mmhr_Sum'] + Potohar['Feb_Precip_above15mmhr_Sum']+ Potohar['Mar_Precip_above15mmhr_Sum']

Potohar['Feb_Mar_Summonthly_Precip_above15mmhr_Sum']= Potohar['Feb_Precip_above15mmhr_Sum']+ Potohar['Mar_Precip_above15mmhr_Sum']




#aggregated precipitation intensity hours below 8mmhr
Potohar['Nov_Dec_Summonthly_Precip_below8mmhr_Sum']= Potohar['Nov_Precip_below8mmhr_Sum']+ Potohar['Dec_Precip_below8mmhr_Sum']
Potohar['Nov_Jan_Summonthly_Precip_below8mmhr_Sum']= Potohar['Nov_Precip_below8mmhr_Sum']+ Potohar['Dec_Precip_below8mmhr_Sum']+Potohar['Jan_Precip_below8mmhr_Sum'] 
Potohar['Nov_Feb_Summonthly_Precip_below8mmhr_Sum']= Potohar['Nov_Precip_below8mmhr_Sum']+ Potohar['Dec_Precip_below8mmhr_Sum']+Potohar['Jan_Precip_below8mmhr_Sum'] + Potohar['Feb_Precip_below8mmhr_Sum']
Potohar['Nov_Feb_Summonthly_Precip_below8mmhr_Sum']= Potohar['Nov_Precip_below8mmhr_Sum']+ Potohar['Dec_Precip_below8mmhr_Sum']+Potohar['Jan_Precip_below8mmhr_Sum'] + Potohar['Feb_Precip_below8mmhr_Sum']+Potohar['Mar_Precip_below8mmhr_Sum']
Potohar['Season_Precip_below8mmhr_Sum_mm']= Potohar['Nov_Precip_below8mmhr_Sum']+ Potohar['Dec_Precip_below8mmhr_Sum']+Potohar['Jan_Precip_below8mmhr_Sum'] + Potohar['Feb_Precip_below8mmhr_Sum']+Potohar['Mar_Precip_below8mmhr_Sum']+Potohar['Apr_Precip_below8mmhr_Sum']

Potohar['Jan_Feb_Summonthly_Precip_below8mmhr_Sum']= Potohar['Jan_Precip_below8mmhr_Sum'] + Potohar['Feb_Precip_below8mmhr_Sum']
Potohar['Jan_Mar_Summonthly_Precip_below8mmhr_Sum']= Potohar['Jan_Precip_below8mmhr_Sum'] + Potohar['Feb_Precip_below8mmhr_Sum']+ Potohar['Mar_Precip_below8mmhr_Sum']

Potohar['Feb_Mar_Summonthly_Precip_below8mmhr_Sum']= Potohar['Feb_Precip_below8mmhr_Sum']+ Potohar['Mar_Precip_below8mmhr_Sum']


#aggregated precipitation intensity hours below 12mmhr
Potohar['Nov_Dec_Summonthly_Precip_below12mmhr_Sum']= Potohar['Nov_Precip_below12mmhr_Sum']+ Potohar['Dec_Precip_below12mmhr_Sum']
Potohar['Nov_Jan_Summonthly_Precip_below12mmhr_Sum']= Potohar['Nov_Precip_below12mmhr_Sum']+ Potohar['Dec_Precip_below12mmhr_Sum']+Potohar['Jan_Precip_below12mmhr_Sum'] 
Potohar['Nov_Feb_Summonthly_Precip_below12mmhr_Sum']= Potohar['Nov_Precip_below12mmhr_Sum']+ Potohar['Dec_Precip_below12mmhr_Sum']+Potohar['Jan_Precip_below12mmhr_Sum'] + Potohar['Feb_Precip_below12mmhr_Sum']
Potohar['Nov_Feb_Summonthly_Precip_below12mmhr_Sum']= Potohar['Nov_Precip_below12mmhr_Sum']+ Potohar['Dec_Precip_below12mmhr_Sum']+Potohar['Jan_Precip_below12mmhr_Sum'] + Potohar['Feb_Precip_below12mmhr_Sum']+Potohar['Mar_Precip_below12mmhr_Sum']
Potohar['Season_Precip_below12mmhr_Sum_mm']= Potohar['Nov_Precip_below12mmhr_Sum']+ Potohar['Dec_Precip_below12mmhr_Sum']+Potohar['Jan_Precip_below12mmhr_Sum'] + Potohar['Feb_Precip_below12mmhr_Sum']+Potohar['Mar_Precip_below12mmhr_Sum']+Potohar['Apr_Precip_below12mmhr_Sum']

Potohar['Jan_Feb_Summonthly_Precip_below12mmhr_Sum']= Potohar['Jan_Precip_below12mmhr_Sum'] + Potohar['Feb_Precip_below12mmhr_Sum']
Potohar['Jan_Mar_Summonthly_Precip_below12mmhr_Sum']= Potohar['Jan_Precip_below12mmhr_Sum'] + Potohar['Feb_Precip_below12mmhr_Sum']+ Potohar['Mar_Precip_below12mmhr_Sum']

Potohar['Feb_Mar_Summonthly_Precip_below12mmhr_Sum']= Potohar['Feb_Precip_below12mmhr_Sum']+ Potohar['Mar_Precip_below12mmhr_Sum']


#aggregated precipitation intensity hours below 15mmhr
Potohar['Nov_Dec_Summonthly_Precip_below15mmhr_Sum']= Potohar['Nov_Precip_below15mmhr_Sum']+ Potohar['Dec_Precip_below15mmhr_Sum']
Potohar['Nov_Jan_Summonthly_Precip_below15mmhr_Sum']= Potohar['Nov_Precip_below15mmhr_Sum']+ Potohar['Dec_Precip_below15mmhr_Sum']+Potohar['Jan_Precip_below15mmhr_Sum'] 
Potohar['Nov_Feb_Summonthly_Precip_below15mmhr_Sum']= Potohar['Nov_Precip_below15mmhr_Sum']+ Potohar['Dec_Precip_below15mmhr_Sum']+Potohar['Jan_Precip_below15mmhr_Sum'] + Potohar['Feb_Precip_below15mmhr_Sum']
Potohar['Nov_Feb_Summonthly_Precip_below15mmhr_Sum']= Potohar['Nov_Precip_below15mmhr_Sum']+ Potohar['Dec_Precip_below15mmhr_Sum']+Potohar['Jan_Precip_below15mmhr_Sum'] + Potohar['Feb_Precip_below15mmhr_Sum']+Potohar['Mar_Precip_below15mmhr_Sum']
Potohar['Season_Precip_below15mmhr_Sum_mm']= Potohar['Nov_Precip_below15mmhr_Sum']+ Potohar['Dec_Precip_below15mmhr_Sum']+Potohar['Jan_Precip_below15mmhr_Sum'] + Potohar['Feb_Precip_below15mmhr_Sum']+Potohar['Mar_Precip_below15mmhr_Sum']+Potohar['Apr_Precip_below15mmhr_Sum']

Potohar['Jan_Feb_Summonthly_Precip_below15mmhr_Sum']= Potohar['Jan_Precip_below15mmhr_Sum'] + Potohar['Feb_Precip_below15mmhr_Sum']
Potohar['Jan_Mar_Summonthly_Precip_below15mmhr_Sum']= Potohar['Jan_Precip_below15mmhr_Sum'] + Potohar['Feb_Precip_below15mmhr_Sum']+ Potohar['Mar_Precip_below15mmhr_Sum']

Potohar['Feb_Mar_Summonthly_Precip_below15mmhr_Sum']= Potohar['Feb_Precip_below15mmhr_Sum']+ Potohar['Mar_Precip_below15mmhr_Sum']



#Do a simple column reorder, and save
wheat = Potohar['Wheat_ton_ha']
district = Potohar['District']

del Potohar['District']
del Potohar ['Wheat_ton_ha']

Potohar.insert(loc=0, column='District', value=district)
Potohar.insert(loc=0, column='Wheat_ton_ha', value=wheat)
Potohar.to_excel('InputData/PotoharDF.xlsx')